In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import sql
from pyspark.sql.types import IntegerType

from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.feature import Normalizer

import pyspark.sql.functions as psf
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Normalizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix



conf = SparkConf().setAppName("milkandbanana").setMaster("local")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)

#conf.set("spark.app.name", "milkandbanana Spark RDD app") 

In [101]:
from pyspark.sql.functions import sqrt

In [3]:
sc.stop()

In [3]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.history.fs.cleaner.interval', '7d'),
 ('spark.shuffle.io.serverThreads', '128'),
 ('spark.sql.streaming.streamingQueryListeners', ''),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.shuffle.file.buffer', '1m'),
 ('spark.sql.hive.convertMetastoreOrc', 'true'),
 ('spark.yarn.dist.files', ''),
 ('spark.sql.autoBroadcastJoinThreshold', '26214400'),
 ('spark.jars',
  'file:///data/home/viktorya.mukukenova/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.11-2.4.5.jar,file:///data/home/viktorya.mukukenova/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.

# Part I

## load dataset

In [5]:
!hdfs dfs -ls /labs/lab06data/ml-100k/u.data

-rw-r--r--   2 hdfs hdfs    1979173 2020-09-30 12:16 /labs/lab06data/ml-100k/u.data


In [6]:
!hdfs dfs -ls /labs/lab06data/ml-100k/u.item

-rw-r--r--   2 hdfs hdfs     236344 2020-09-30 12:16 /labs/lab06data/ml-100k/u.item


In [17]:
rdd_data = sc.textFile("/labs/lab06data/ml-100k/u.data")
rdd_item = sc.textFile("/labs/lab06data/ml-100k/u.item")

In [104]:
rdd_data.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [105]:
rdd_item.take(10)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0',
 '6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0',
 '8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title

In [22]:
data = rdd_data.map(lambda x: x.split("\t"))\
.toDF()

data.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)



In [83]:
data.filter(data._2 == 234)\
.groupBy("_3").count().show()

+---+-----+
| _3|count|
+---+-----+
|  3|   69|
|  5|   68|
|  1|    8|
|  4|  116|
|  2|   19|
+---+-----+



In [90]:
data\
.groupBy("_3").count()\
.show()

+---+-----+
| _3|count|
+---+-----+
|  3|27145|
|  5|21201|
|  1| 6110|
|  4|34174|
|  2|11370|
+---+-----+

